# Information

**Author:**<br>Pascal Munaretto (<a href="mailto:pascal.munaretto@outlook.com">Mail</a>)

**Date:**<br>30.09.2022

**Type:**<br>Master's Thesis

**Topic:**<br>Design, Implementation and Performance Analysis of an AI-Based Insider Threat Detection Platform	in Splunk To Counteract Data Exfiltration

**Study Program:**<br>Enterprise and IT Security

**Institution:**<br><a href="https://www.hs-offenburg.de">Offenburg University of Applied Sciences</a>

**Github:**<br>https://github.com/pmunaretto/Master-Thesis

# Setup

## Requirements

In [ ]:
!pip install pyod suod

## Patches

In [ ]:
# Add callbacks to Auto Encoder, VAE and Deep SVDD
!cp /content/drive/MyDrive/CERT/patches/patched_auto_encoder.py /usr/local/lib/python3.7/dist-packages/pyod/models/auto_encoder.py
!cp /content/drive/MyDrive/CERT/patches/patched_vae.py /usr/local/lib/python3.7/dist-packages/pyod/models/vae.py
!cp /content/drive/MyDrive/CERT/patches/patched_deep_svdd.py /usr/local/lib/python3.7/dist-packages/pyod/models/deep_svdd.py

## Imports

In [ ]:
import os
import math
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from timeit import default_timer as timer
from random import seed, randint
from sklearn.base import TransformerMixin, BaseEstimator, clone
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, recall_score, f1_score, precision_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import make_column_selector as selector
from matplotlib.backends.backend_pgf import FigureCanvasPgf
from matplotlib.ticker import PercentFormatter
from pyod.models.iforest import IForest
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.loda import LODA
from pyod.models.cblof import CBLOF
from pyod.models.pca import PCA
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.deep_svdd import DeepSVDD
from IPython.display import display, Markdown

## Configuration

In [ ]:
matplotlib.backend_bases.register_backend("pgf", FigureCanvasPgf)

plt.rcParams.update({
    "figure.dpi": 100,
    "savefig.dpi": 300,
    "font.size": 12,
    "image.cmap": "plasma",
    "axes.prop_cycle": plt.cycler("color", "bgrcmyk"), 
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False
})

tf.get_logger().setLevel("WARN")

# Global Configuration
BASE_PATH     = "/content/drive/MyDrive/CERT/r4.2"
N_JOBS        = -1
N_ITER        = 5
CONTAMINATION = 0.01
RETRAIN       = False
DATASET_NAME  = "logon_sessions_role"
FEATURE_SETS  = [
    ["session_duration", "hour"],
    ["session_duration", "weekday"],
    ["session_duration", "hour", "weekday"],
    ["session_duration", "hour_sin", "hour_cos"],
    ["session_duration", "weekday_sin", "weekday_cos"],
    ["session_duration", "hour_sin", "hour_cos", "weekday_sin", "weekday_cos"]
]

## Helper Functions

In [ ]:
class Debugger(BaseEstimator, TransformerMixin):

    def transform(self, data):
        print("Shape of Preprocessed data:", data.shape)
        print(pd.DataFrame(data).head())
        return data

    def fit(self, data, y=None, **fit_params):
        return self


def plot_anomaly_scores(series, identifier, min, max, save=True):
    plt.figure(figsize=(10,3))
    plt.hist(
        series,
        weights=np.ones(len(series)) / len(series),
        bins=np.arange(min, max, 0.02),
        rwidth=0.8
    )
    plt.xlim(xmin=min, xmax=max)
    plt.xticks(np.arange(min, max+0.1, 0.1))
    plt.xlabel("Anomaly Score")
    plt.ylabel("Percentage")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def plot_confusion_matrix(y_true, y_pred, identifier, save=True):
    ConfusionMatrixDisplay.from_predictions(
        y_true,
        y_pred,
        labels=[0, 1],
        display_labels=["Benign", "Malicious"],
        values_format="d",
        colorbar=True,
        cmap="plasma_r"
    )
    plt.grid(False)
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def print_training_result(metrics):
    print(
        "  ".join(
            [
                f"\033[1;33m Training Time: {metrics.training_time_avg:<7.4f}\033[0m",
                f"\033[1;33m Inference Time: {metrics.inference_time_avg:<7.4f}\033[0m",
                f"\033[1;35m pAUC: {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}\033[0m",
                f"\033[1;35m Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}\033[0m",
                f"\033[1;32m TN: {metrics.best_classifier_TN:<6}\033[0m",
                f"\033[1;31m FP: {metrics.best_classifier_FP:<5}\033[0m",
                f"\033[1;31m FN: {metrics.best_classifier_FN:<3}\033[0m",
                f"\033[1;32m TP: {metrics.best_classifier_TP:<3}\033[0m",
                f"\033[1;37m Params: {metrics.name}\033[0m"
            ]
        )
    )


def print_gridsearch_result(metrics):
    print(
        "\n".join(
            [
                "\n\033[4mBest hyperparameters:\033[0m",
                f"Params: {metrics.name}",
                f"pAUC:   {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}",
                f"Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}"
            ]
        )
    )


def calculate_dispersion_metrics_for_columns(source_df, destination_df, columns):
    for column in columns:
        avg = np.average(source_df[column])
        std = np.std(source_df[column])
        destination_df[f"{column}_avg"] = avg
        destination_df[f"{column}_std"] = std if not math.isnan(std) else 0

    return destination_df


def train_classifier_on_single_users(df, classifier, features, params, n_iter=10):
    # Create a dataframe where the results of the different seeds will be stored
    random_state_summary = pd.DataFrame()

    # Reset the PRNG seed
    seed(1)

    # Group the dataframe by users
    grouped = df.groupby("functional_unit", as_index=False)

    # Perform the training process multiple times with random seeds
    for _ in range(n_iter):

        # Create a dataframe where the results of the classifiers will be stored
        user_timings = pd.DataFrame()
        user_predictions = pd.DataFrame()

        for name, group in grouped:

            # Create a clone of the classifier
            try:
                classifier = clone(classifier)
            except:
                pass
            
            # Update the parameters of the classifier according to the grid search
            classifier.set_params(**params)
            
            # Set the random state attribute of the classifier (if it has one)
            try:
                classifier.set_params(**{"random_state": randint(0, 2**32)})
            except Exception:
                pass

            # Define the transformers that do the rest of the preprocessing (scaling, encoding)
            numeric_transformer = Pipeline(steps=[
                ("scaler", RobustScaler())
            ])
            categorical_transformer = Pipeline(steps=[
                ("ohe", OneHotEncoder())
            ])

            # Filter the dataframe according to the features 
            group_filtered = group[features].copy()

            # Create a pipeline that performs the feature selection and scaling
            pipe = Pipeline([
                ("column_transformer", ColumnTransformer(
                    transformers=[
                        ("num", numeric_transformer, selector(dtype_exclude=["category", "object"])),
                        ("cat", categorical_transformer, selector(dtype_include=["category", "object"]))
                    ]
                )),
                ("classifier", classifier)
            ])
        
            # Benchmark the training
            start_training = timer()
            pipe.fit(group_filtered)
            end_training = timer()

            # Benchmark the inference
            start_inference = timer()
            pipe.predict(group_filtered)
            end_inference = timer()

            # Add the predictions and anomaly scores to the group dataframe
            group["y_true"] = group["threat"]
            group["y_pred"] = pipe.named_steps["classifier"].labels_
            group["scores"] = pipe.named_steps["classifier"].decision_scores_

            # Create a new series with the training metrics for the user iteration
            timings = pd.Series(
                {
                    "training_time": end_training - start_training,
                    "inference_time": end_inference - start_inference
                }
            )

            # Append the series to our user summary dataframes
            user_timings = user_timings.append(timings, ignore_index=True)
            user_predictions = pd.concat([user_predictions, group])

        # If there are inf values in the anomaly scores, replace it with the elsewise highest value
        max_without_infs = user_predictions["scores"].replace([np.inf, -np.inf], np.nan).max()
        user_predictions["scores"].replace(np.inf, max_without_infs, inplace=True)

        # Find the right threshold to satisfy the contamination and add the final predictions
        upper = max_without_infs
        lower = 0
        for i in range(0,500):
            threshold = (upper + lower) / 2
            test = (user_predictions.scores > threshold).astype(bool)
            if test.sum() > CONTAMINATION * len(df):
                lower = threshold
            else:
                upper = threshold
        user_predictions["y_pred"] = test

        # After the predictions were added, calculate the evaluation metrics
        recall = recall_score(user_predictions.threat, user_predictions.y_pred)
        precision = precision_score(user_predictions.threat, user_predictions.y_pred)
        f1 = f1_score(user_predictions.threat, user_predictions.y_pred)
        cm = confusion_matrix(user_predictions.threat, user_predictions.y_pred, labels=[0, 1])

        # Try to calculate the AUC, this could fail if one of the anomaly scores is infinite
        try:
            auc = roc_auc_score(user_predictions.threat, user_predictions.scores)
            p_auc_10 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.1)
            p_auc_20 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.2)
            p_auc_30 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.3)
        except ValueError as e:
            print(e)

        # Create a new series with all the information about the iteration
        metrics = pd.Series(
            {
                "training_time": user_timings.training_time.sum(),
                "inference_time": user_timings.inference_time.sum(),
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "TN": cm[0][0],
                "FP": cm[0][1],
                "FN": cm[1][0],
                "TP": cm[1][1],
                "auc": auc,
                "p_auc_10": p_auc_10,
                "p_auc_20": p_auc_20,
                "p_auc_30": p_auc_30,
                "y_true": user_predictions.y_true,
                "y_pred": user_predictions.y_pred,
                "scores": user_predictions.scores,
            }
        )

        # Append the series to our summary dataframe
        random_state_summary = random_state_summary.append(metrics, ignore_index=True)

    # Convert the confusion matrix to integers
    random_state_summary = random_state_summary.astype({"TN": "int32", "FP": "int32", "FN": "int32", "TP": "int32"})

    # Locate the best classifier and separate the predictions from it
    results = random_state_summary.iloc[random_state_summary.p_auc_10.argmax()].rename(str(params))
    predictions = results.loc[["y_true", "y_pred", "scores"]]

    # Remove the columns that should not be part of the results dataframe
    results.drop(["y_true", "y_pred", "scores"], inplace=True)

    # Add the prefix
    results = results.add_prefix("best_classifier_")
    
    # Add the average training and inference time to the dataframe
    results["training_time_avg"]  = np.average(random_state_summary["training_time"])
    results["inference_time_avg"] = np.average(random_state_summary["inference_time"])

    # Calculate averages and different dispersion metrics for the best classifier series
    results = calculate_dispersion_metrics_for_columns(
        source_df=random_state_summary,
        destination_df=results,
        columns=["auc", "p_auc_10", "p_auc_20", "p_auc_30", "recall"]
    )

    return results, predictions


class GridSearch:
    def __init__(self, df, classifier, features, parameters, gridsearch_path):

        # Instance variables
        self.df = df
        self.classifier = classifier
        self.features = features
        self.parameters = parameters
        self.gridsearch_path = gridsearch_path

        # Main paths
        self.summary_path = os.path.join(self.gridsearch_path, "gridsearch_summary.csv")
        self.best_results_path = os.path.join(self.gridsearch_path, "best_results.csv")
        self.best_preds_path = os.path.join(self.gridsearch_path, "best_preds.csv")

        # Create the output directory for the gridsearch
        os.makedirs(gridsearch_path, exist_ok=True)

        # Read existing files
        if os.path.exists(self.summary_path) and not RETRAIN:
            self.gridsearch_summary = pd.read_csv(self.summary_path, index_col=0)
        else:
            self.gridsearch_summary = pd.DataFrame()
        if os.path.exists(self.best_results_path) and not RETRAIN:
            self.best_results = pd.read_csv(self.best_results_path, squeeze=True, index_col=0)
        else:
            self.best_results = None


    def start_training(self):

        # Create an iterable parameter grid from the parameters dictionary
        grid = ParameterGrid(self.parameters)

        # Debug output
        print(f"\033[4mTesting {len(list(grid))} different hyperparameter combinations\033[0m")

        # Iterate over all possible parameter combinations
        for params in grid:

            # Skip the parameters if they are already part of the gridsearch summary
            if not RETRAIN and not self.gridsearch_summary.empty and str(params) in self.gridsearch_summary.index:
                print_training_result(self.gridsearch_summary.loc[str(params)])
                continue

            # Start the training process
            try: 
                if hasattr(classifier, "random_state"):
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        features=self.features,
                        params=params,
                        n_iter=N_ITER
                    )
                else:
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        features=self.features,
                        params=params,
                        n_iter=1
                    )
            except ValueError as e:
                print(f"Skipping {params}: {e}")
                continue

            # Print the metrics of the best classifier
            print_training_result(results)

            # Add the results to the gridsearch summary
            self.gridsearch_summary = self.gridsearch_summary.append(results)

            # Update the best classifier if the iterations performs better than the current best 
            if self.best_results is None or results.p_auc_10_avg > self.best_results.p_auc_10_avg:
                self.best_results = results
                self.best_preds = predictions
                self.save_best_results()

            # Save the progress
            self.save_gridsearch_summary()

        # Print the results of the gridsearch (parameters with the best average)
        print_gridsearch_result(self.best_results)

        return self.best_results


    def save_gridsearch_summary(self):
        self.gridsearch_summary.to_csv(self.summary_path)


    def save_best_results(self):
        self.best_results.to_csv(self.best_results_path)
        self.best_preds.to_frame()\
            .transpose()\
            .apply(pd.Series.explode)\
            .reset_index(drop=True)\
            .to_csv(self.best_preds_path, index=False)


    def get_summary(self):
        return self.gridsearch_summary


def initiate_training_run(classifier_name, classifier, parameters):
    # Define output paths
    summary_path = os.path.join(BASE_PATH, "results_summary", DATASET_NAME)
    summary_file = os.path.join(summary_path, "summary.csv")

    # Create the output directory for the classifier
    os.makedirs(summary_path, exist_ok=True)

    # Iterate through the feature sets
    for i, features in enumerate(FEATURE_SETS, start=1):

        display(Markdown(f"# {i}/{len(FEATURE_SETS)} - Features: {', '.join(features)}"))

        # Perform a grid search to find the best parameters for the classifier
        gridsearch = GridSearch(
            df=df,
            classifier=classifier,
            features=features,
            parameters=parameters,
            gridsearch_path=os.path.join(BASE_PATH, "results_summary", DATASET_NAME, classifier_name, f"gridsearch{i}")
        )
    
        best_parameter_series = gridsearch.start_training()

        # Read the summary file if it already exists, otherwise create a new one
        if os.path.exists(summary_file):
            summary = pd.read_csv(summary_file, index_col=0)
        else:
            summary = pd.DataFrame()

        # Set the index of the pandas series and update / append it to the summary
        index_name = f"{classifier_name}_dataset{i}"
        best_parameter_series.rename(index_name, inplace=True)
        if index_name in summary.index:
            summary.loc[index_name] = best_parameter_series
        else:
            summary = summary.append(best_parameter_series)

        # Save the summaries and predictions to a file
        summary.sort_index(inplace=True)
        summary.to_csv(summary_file)

## Loading the Data

In [ ]:
# Read the dataset
df = pd.read_parquet(os.path.join(BASE_PATH, "preprocessed", "logon_sessions"))

# Training - Isolation Forest

In [ ]:
# Local configuration
classifier_name = "isolation_forest"

# Define the classifier that will be used for training
classifier = IForest(
    behaviour="new",
    max_features=1.0,
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_estimators": [1, 10, 50, 100],
    "max_samples": [128, 256, 512, 1024, 2048, 4096],
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 24 different hyperparameter combinations
 Training Time: 0.6088    Inference Time: 0.2478    pAUC: 0.6053 ± 0.0529   Recall: 0.0727 ± 0.0924   TN: 467454   FP: 3038    FN: 75    TP: 24    Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 3.1436    Inference Time: 1.1690    pAUC: 0.7411 ± 0.0165   Recall: 0.2263 ± 0.0942   TN: 466073   FP: 4419    FN: 58    TP: 41    Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 11.0406   Inference Time: 4.8451    pAUC: 0.7612 ± 0.0149   Recall: 0.2707 ± 0.0569   TN: 465959   FP: 4533    FN: 62    TP: 37    Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 21.5671   Inference Time: 9.9430    pAUC: 0.7739 ± 0.0107   Recall: 0.3030 ± 0.0546   TN: 465887   FP: 4605    FN: 63    TP: 36    Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.4121    Inference Time: 0.1658    pAUC: 0.6596 ± 0.0342   Recall: 0.1212 ± 0.0221   TN: 468889   FP: 1603    FN: 87    TP: 12    Params: {'max_sample

# 2/6 - Features: session_duration, weekday

Testing 24 different hyperparameter combinations
 Training Time: 0.3934    Inference Time: 0.1592    pAUC: 0.5317 ± 0.0151   Recall: 0.0000 ± 0.0000   TN: 469124   FP: 1368    FN: 99    TP: 0     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 2.8767    Inference Time: 1.0586    pAUC: 0.5696 ± 0.0152   Recall: 0.0283 ± 0.0118   TN: 466724   FP: 3768    FN: 94    TP: 5     Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 11.0994   Inference Time: 4.9565    pAUC: 0.5775 ± 0.0089   Recall: 0.0343 ± 0.0049   TN: 466000   FP: 4492    FN: 95    TP: 4     Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 21.9026   Inference Time: 9.9408    pAUC: 0.5787 ± 0.0065   Recall: 0.0343 ± 0.0049   TN: 465833   FP: 4659    FN: 95    TP: 4     Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.4064    Inference Time: 0.1669    pAUC: 0.5321 ± 0.0194   Recall: 0.0040 ± 0.0049   TN: 468689   FP: 1803    FN: 98    TP: 1     Params: {'max_sample

# 3/6 - Features: session_duration, hour, weekday

...

Testing 24 different hyperparameter combinations
 Training Time: 0.4091    Inference Time: 0.1624    pAUC: 0.6291 ± 0.0575   Recall: 0.0687 ± 0.0650   TN: 468635   FP: 1857    FN: 81    TP: 18    Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 2.8384    Inference Time: 1.0283    pAUC: 0.7812 ± 0.0310   Recall: 0.2505 ± 0.0590   TN: 465985   FP: 4507    FN: 71    TP: 28    Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 10.8557   Inference Time: 4.8105    pAUC: 0.8226 ± 0.0063   Recall: 0.3515 ± 0.0134   TN: 466023   FP: 4469    FN: 64    TP: 35    Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 21.4593   Inference Time: 9.7895    pAUC: 0.8230 ± 0.0097   Recall: 0.3434 ± 0.0278   TN: 465870   FP: 4622    FN: 65    TP: 34    Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.4219    Inference Time: 0.1710    pAUC: 0.6998 ± 0.0388   Recall: 0.1960 ± 0.0909   TN: 468189   FP: 2303    FN: 67    TP: 32    Params: {'max_sample

# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 24 different hyperparameter combinations
 Training Time: 0.4130    Inference Time: 0.1668    pAUC: 0.5590 ± 0.0128   Recall: 0.0020 ± 0.0040   TN: 470483   FP: 9       FN: 99    TP: 0     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 2.8931    Inference Time: 1.0574    pAUC: 0.5803 ± 0.0113   Recall: 0.0343 ± 0.0081   TN: 465964   FP: 4528    FN: 95    TP: 4     Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 11.0194   Inference Time: 4.8899    pAUC: 0.5954 ± 0.0033   Recall: 0.0384 ± 0.0076   TN: 465899   FP: 4593    FN: 96    TP: 3     Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 21.3576   Inference Time: 9.5623    pAUC: 0.5936 ± 0.0035   Recall: 0.0404 ± 0.0000   TN: 465845   FP: 4647    FN: 95    TP: 4     Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.4360    Inference Time: 0.1731    pAUC: 0.5554 ± 0.0237   Recall: 0.0040 ± 0.0049   TN: 468403   FP: 2089    FN: 99    TP: 0     Params: {'max_sample

# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 24 different hyperparameter combinations
 Training Time: 0.4538    Inference Time: 0.1794    pAUC: 0.6107 ± 0.0344   Recall: 0.0606 ± 0.0971   TN: 470410   FP: 82      FN: 99    TP: 0     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 2.9826    Inference Time: 1.0970    pAUC: 0.7548 ± 0.0279   Recall: 0.1697 ± 0.0242   TN: 466277   FP: 4215    FN: 80    TP: 19    Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 11.7304   Inference Time: 5.1275    pAUC: 0.8083 ± 0.0221   Recall: 0.2121 ± 0.0452   TN: 465821   FP: 4671    FN: 77    TP: 22    Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 23.0078   Inference Time: 10.2700   pAUC: 0.8248 ± 0.0117   Recall: 0.2061 ± 0.0121   TN: 465831   FP: 4661    FN: 78    TP: 21    Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.4823    Inference Time: 0.1889    pAUC: 0.6708 ± 0.0611   Recall: 0.0707 ± 0.0899   TN: 468830   FP: 1662    FN: 95    TP: 4     Params: {'max_sample

# Training - LODA

In [ ]:
# Local configuration
classifier_name = "loda"

# Define the classifier that will be used for training
classifier = LODA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_bins": [6, 8, 10, 12, 14, 16, 20],
    "n_random_cuts": [25, 50, 75, 100]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 28 different hyperparameter combinations
 Training Time: 1.1212    Inference Time: 0.6754    pAUC: 0.7196 ± 0.0000   Recall: 0.2626 ± 0.0000   TN: 467020   FP: 3472    FN: 73    TP: 26    Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 2.1723    Inference Time: 1.3784    pAUC: 0.6773 ± 0.0000   Recall: 0.2323 ± 0.0000   TN: 467703   FP: 2789    FN: 76    TP: 23    Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 3.2367    Inference Time: 1.9984    pAUC: 0.7401 ± 0.0000   Recall: 0.2727 ± 0.0000   TN: 467072   FP: 3420    FN: 72    TP: 27    Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 4.1402    Inference Time: 2.6219    pAUC: 0.7274 ± 0.0000   Recall: 0.4343 ± 0.0000   TN: 465773   FP: 4719    FN: 56    TP: 43    Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 1.1677    Inference Time: 0.7239    pAUC: 0.7125 ± 0.0000   Recall: 0.4545 ± 0.0000   TN: 465678   FP: 4814    FN: 54    TP: 45    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 2/6 - Features: session_duration, weekday

Testing 28 different hyperparameter combinations
 Training Time: 1.1454    Inference Time: 0.6867    pAUC: 0.5428 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 464025   FP: 6467    FN: 99    TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 2.2440    Inference Time: 1.3323    pAUC: 0.5413 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465697   FP: 4795    FN: 99    TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 3.2512    Inference Time: 1.9358    pAUC: 0.5298 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465828   FP: 4664    FN: 99    TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 4.3049    Inference Time: 2.5333    pAUC: 0.5147 ± 0.0000   Recall: 0.0606 ± 0.0000   TN: 465949   FP: 4543    FN: 93    TP: 6     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 1.1772    Inference Time: 0.7109    pAUC: 0.4890 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 465101   FP: 5391    FN: 97    TP: 2     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 3/6 - Features: session_duration, hour, weekday

...

Testing 28 different hyperparameter combinations
 Training Time: 1.1723    Inference Time: 0.6780    pAUC: 0.7329 ± 0.0000   Recall: 0.3838 ± 0.0000   TN: 465093   FP: 5399    FN: 61    TP: 38    Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 2.2162    Inference Time: 1.3575    pAUC: 0.7213 ± 0.0000   Recall: 0.2929 ± 0.0000   TN: 464788   FP: 5704    FN: 70    TP: 29    Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 3.2174    Inference Time: 2.0069    pAUC: 0.7321 ± 0.0000   Recall: 0.2929 ± 0.0000   TN: 465837   FP: 4655    FN: 70    TP: 29    Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 4.2431    Inference Time: 2.6065    pAUC: 0.7368 ± 0.0000   Recall: 0.3434 ± 0.0000   TN: 465810   FP: 4682    FN: 65    TP: 34    Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 1.2176    Inference Time: 0.7290    pAUC: 0.7941 ± 0.0000   Recall: 0.4646 ± 0.0000   TN: 465827   FP: 4665    FN: 53    TP: 46    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 28 different hyperparameter combinations
 Training Time: 1.1508    Inference Time: 0.6609    pAUC: 0.5470 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465458   FP: 5034    FN: 99    TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 2.1896    Inference Time: 1.2551    pAUC: 0.4980 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 466993   FP: 3499    FN: 96    TP: 3     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 3.1962    Inference Time: 1.8930    pAUC: 0.5132 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 467938   FP: 2554    FN: 96    TP: 3     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 4.1541    Inference Time: 2.5236    pAUC: 0.5011 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 467389   FP: 3103    FN: 98    TP: 1     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 1.1542    Inference Time: 0.6782    pAUC: 0.5283 ± 0.0000   Recall: 0.0606 ± 0.0000   TN: 465783   FP: 4709    FN: 93    TP: 6     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 28 different hyperparameter combinations
 Training Time: 1.2015    Inference Time: 0.6814    pAUC: 0.4902 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 465462   FP: 5030    FN: 97    TP: 2     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 2.1936    Inference Time: 1.3440    pAUC: 0.5496 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 467565   FP: 2927    FN: 96    TP: 3     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 3.2148    Inference Time: 1.9881    pAUC: 0.5429 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 465787   FP: 4705    FN: 96    TP: 3     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 4.2389    Inference Time: 2.6126    pAUC: 0.5377 ± 0.0000   Recall: 0.0808 ± 0.0000   TN: 465822   FP: 4670    FN: 91    TP: 8     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 1.2333    Inference Time: 0.7075    pAUC: 0.5142 ± 0.0000   Recall: 0.0606 ± 0.0000   TN: 465917   FP: 4575    FN: 93    TP: 6     Params: {'n_bins': 8, 'n_random_cuts': 25}


# Training - COPOD

In [ ]:
# Local configuration
classifier_name = "copod"

# Define the classifier that will be used for training
classifier = COPOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# Training - ECOD

In [ ]:
# Local configuration
classifier_name = "ecod"

# Define the classifier that will be used for training
classifier = ECOD(
    contamination=CONTAMINATION,
    n_jobs=1
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 1 different hyperparameter combinations
 Training Time: 0.4621    Inference Time: 0.8196    pAUC: 0.7568 ± 0.0000   Recall: 0.4141 ± 0.0000   TN: 465825   FP: 4667    FN: 58    TP: 41    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7568 ± 0.0000
Recall: 0.4141 ± 0.0000


# 2/6 - Features: session_duration, weekday

Testing 1 different hyperparameter combinations
 Training Time: 0.4657    Inference Time: 0.7841    pAUC: 0.5134 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 465790   FP: 4702    FN: 96    TP: 3     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5134 ± 0.0000
Recall: 0.0303 ± 0.0000


# 3/6 - Features: session_duration, hour, weekday

...

Testing 1 different hyperparameter combinations
 Training Time: 0.6394    Inference Time: 1.0903    pAUC: 0.7538 ± 0.0000   Recall: 0.2424 ± 0.0000   TN: 465811   FP: 4681    FN: 75    TP: 24    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7538 ± 0.0000
Recall: 0.2424 ± 0.0000


# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 0.5747    Inference Time: 1.0222    pAUC: 0.5056 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465787   FP: 4705    FN: 99    TP: 0     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5056 ± 0.0000
Recall: 0.0000 ± 0.0000


# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 0.8712    Inference Time: 1.5687    pAUC: 0.7482 ± 0.0000   Recall: 0.2020 ± 0.0000   TN: 465806   FP: 4686    FN: 79    TP: 20    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7482 ± 0.0000
Recall: 0.2020 ± 0.0000


# Training - CBLOF

In [ ]:
# Local configuration
classifier_name = "cblof"

# Define the classifier that will be used for training
classifier = CBLOF(
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_clusters": [2, 4],
    "alpha": [0.2, 0.4, 0.6, 0.8, 0.9],
    "beta": [2, 4, 8, 16],
    "use_weights": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 80 different hyperparameter combinations
 Training Time: 2.8362    Inference Time: 0.1748    pAUC: 0.7283 ± 0.0000   Recall: 0.3636 ± 0.0000   TN: 465822   FP: 4670    FN: 63    TP: 36    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 2.4189    Inference Time: 0.1452    pAUC: 0.6969 ± 0.0000   Recall: 0.2424 ± 0.0000   TN: 465819   FP: 4673    FN: 75    TP: 24    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 4.7596    Inference Time: 0.1091    pAUC: 0.5626 ± 0.0184   Recall: 0.1455 ± 0.0283   TN: 465829   FP: 4663    FN: 79    TP: 20    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 4.7834    Inference Time: 0.1252    pAUC: 0.7040 ± 0.0013   Recall: 0.2606 ± 0.0242   TN: 465806   FP: 4686    FN: 72    TP: 27    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 2.3773    Inference Time: 0.1512    pAUC: 0.7283 ± 0.0000

# 2/6 - Features: session_duration, weekday

Testing 80 different hyperparameter combinations
 Training Time: 2.6125    Inference Time: 0.1459    pAUC: 0.4889 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465782   FP: 4710    FN: 99    TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 2.6859    Inference Time: 0.1340    pAUC: 0.6099 ± 0.0003   Recall: 0.1313 ± 0.0000   TN: 465800   FP: 4692    FN: 86    TP: 13    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 4.9127    Inference Time: 0.1334    pAUC: 0.5143 ± 0.0028   Recall: 0.0000 ± 0.0000   TN: 465787   FP: 4705    FN: 99    TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 4.6981    Inference Time: 0.1223    pAUC: 0.6138 ± 0.0013   Recall: 0.0424 ± 0.0444   TN: 465758   FP: 4734    FN: 97    TP: 2     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 2.6133    Inference Time: 0.1644    pAUC: 0.4889 ± 0.0000

# 3/6 - Features: session_duration, hour, weekday

...

Testing 80 different hyperparameter combinations
 Training Time: 2.2834    Inference Time: 0.1746    pAUC: 0.6369 ± 0.0000   Recall: 0.1919 ± 0.0000   TN: 465810   FP: 4682    FN: 80    TP: 19    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 2.3522    Inference Time: 0.1673    pAUC: 0.6327 ± 0.0000   Recall: 0.0606 ± 0.0000   TN: 465784   FP: 4708    FN: 93    TP: 6     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 4.1428    Inference Time: 0.1294    pAUC: 0.5180 ± 0.0056   Recall: 0.0606 ± 0.0202   TN: 465808   FP: 4684    FN: 97    TP: 2     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 4.3799    Inference Time: 0.1364    pAUC: 0.6292 ± 0.0011   Recall: 0.0687 ± 0.0040   TN: 465788   FP: 4704    FN: 92    TP: 7     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 2.1791    Inference Time: 0.1726    pAUC: 0.6369 ± 0.0000

# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 80 different hyperparameter combinations
 Training Time: 2.5921    Inference Time: 0.1580    pAUC: 0.5248 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 465526   FP: 4966    FN: 96    TP: 3     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 2.7240    Inference Time: 0.1697    pAUC: 0.5974 ± 0.0000   Recall: 0.1313 ± 0.0000   TN: 465800   FP: 4692    FN: 86    TP: 13    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 4.6333    Inference Time: 0.1282    pAUC: 0.5292 ± 0.0016   Recall: 0.0263 ± 0.0081   TN: 465827   FP: 4665    FN: 96    TP: 3     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 4.3848    Inference Time: 0.1517    pAUC: 0.6117 ± 0.0004   Recall: 0.0263 ± 0.0081   TN: 465838   FP: 4654    FN: 96    TP: 3     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 2.7099    Inference Time: 0.1714    pAUC: 0.5248 ± 0.0000

# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 80 different hyperparameter combinations
 Training Time: 2.8419    Inference Time: 0.1656    pAUC: 0.6008 ± 0.0000   Recall: 0.1919 ± 0.0000   TN: 465806   FP: 4686    FN: 80    TP: 19    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 2.6855    Inference Time: 0.1924    pAUC: 0.6317 ± 0.0000   Recall: 0.0606 ± 0.0000   TN: 465790   FP: 4702    FN: 93    TP: 6     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 5.3892    Inference Time: 0.1506    pAUC: 0.5761 ± 0.0276   Recall: 0.1232 ± 0.0099   TN: 465799   FP: 4693    FN: 86    TP: 13    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 5.0080    Inference Time: 0.1368    pAUC: 0.7138 ± 0.0425   Recall: 0.2343 ± 0.1007   TN: 465823   FP: 4669    FN: 60    TP: 39    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 2.6939    Inference Time: 0.1789    pAUC: 0.6008 ± 0.0000

# Training - PCA

In [ ]:
# Local configuration
classifier_name = "pca"

# Define the classifier that will be used for training
classifier = PCA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_components": [1, 2, 3, 4, 5, 6, 7],
    "whiten": [True, False],
    "svd_solver": ["full", "arpack", "randomized"],
    "weighted": [True, False],
    "standardization": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 168 different hyperparameter combinations
 Training Time: 0.1964    Inference Time: 0.0678    pAUC: 0.7692 ± 0.0000   Recall: 0.4343 ± 0.0000   TN: 465844   FP: 4648    FN: 56    TP: 43    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.1744    Inference Time: 0.0658    pAUC: 0.7692 ± 0.0000   Recall: 0.4343 ± 0.0000   TN: 465844   FP: 4648    FN: 56    TP: 43    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.1786    Inference Time: 0.0674    pAUC: 0.7756 ± 0.0000   Recall: 0.4747 ± 0.0000   TN: 465760   FP: 4732    FN: 52    TP: 47    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.1765    Inference Time: 0.0642    pAUC: 0.7756 ± 0.0000   Recall: 0.4747 ± 0.0000   TN: 465760   FP: 4732    FN: 52    TP: 47    Params: {'n_components': 1, 'stan

# 2/6 - Features: session_duration, weekday

Testing 168 different hyperparameter combinations
 Training Time: 0.1831    Inference Time: 0.0669    pAUC: 0.4878 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 465800   FP: 4692    FN: 98    TP: 1     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.1753    Inference Time: 0.0656    pAUC: 0.4878 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 465800   FP: 4692    FN: 98    TP: 1     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.1723    Inference Time: 0.0624    pAUC: 0.4870 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 465794   FP: 4698    FN: 98    TP: 1     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.1737    Inference Time: 0.0632    pAUC: 0.4870 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 465794   FP: 4698    FN: 98    TP: 1     Params: {'n_components': 1, 'stan

# 3/6 - Features: session_duration, hour, weekday

...

Testing 168 different hyperparameter combinations
 Training Time: 0.2175    Inference Time: 0.0781    pAUC: 0.7807 ± 0.0000   Recall: 0.5152 ± 0.0000   TN: 465794   FP: 4698    FN: 48    TP: 51    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.2186    Inference Time: 0.0775    pAUC: 0.7807 ± 0.0000   Recall: 0.5152 ± 0.0000   TN: 465794   FP: 4698    FN: 48    TP: 51    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.2158    Inference Time: 0.0781    pAUC: 0.7775 ± 0.0000   Recall: 0.5051 ± 0.0000   TN: 465845   FP: 4647    FN: 49    TP: 50    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.2211    Inference Time: 0.0789    pAUC: 0.7775 ± 0.0000   Recall: 0.5051 ± 0.0000   TN: 465845   FP: 4647    FN: 49    TP: 50    Params: {'n_components': 1, 'stan

# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 168 different hyperparameter combinations
 Training Time: 0.2189    Inference Time: 0.0737    pAUC: 0.6140 ± 0.0000   Recall: 0.0606 ± 0.0000   TN: 465817   FP: 4675    FN: 93    TP: 6     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.2142    Inference Time: 0.0741    pAUC: 0.6140 ± 0.0000   Recall: 0.0606 ± 0.0000   TN: 465817   FP: 4675    FN: 93    TP: 6     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.2194    Inference Time: 0.0768    pAUC: 0.6143 ± 0.0000   Recall: 0.0707 ± 0.0000   TN: 465811   FP: 4681    FN: 92    TP: 7     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.2152    Inference Time: 0.0724    pAUC: 0.6143 ± 0.0000   Recall: 0.0707 ± 0.0000   TN: 465811   FP: 4681    FN: 92    TP: 7     Params: {'n_components': 1, 'stan

# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 168 different hyperparameter combinations
 Training Time: 0.2887    Inference Time: 0.0918    pAUC: 0.7803 ± 0.0000   Recall: 0.5152 ± 0.0000   TN: 465838   FP: 4654    FN: 48    TP: 51    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.2822    Inference Time: 0.0888    pAUC: 0.7803 ± 0.0000   Recall: 0.5152 ± 0.0000   TN: 465838   FP: 4654    FN: 48    TP: 51    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.2878    Inference Time: 0.0875    pAUC: 0.7795 ± 0.0000   Recall: 0.5051 ± 0.0000   TN: 465839   FP: 4653    FN: 49    TP: 50    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.2873    Inference Time: 0.0874    pAUC: 0.7795 ± 0.0000   Recall: 0.5051 ± 0.0000   TN: 465839   FP: 4653    FN: 49    TP: 50    Params: {'n_components': 1, 'stan

# Training - AE

In [ ]:
# Local configuration
classifier_name = "auto_encoder"

# Define the classifier that will be used for training
classifier = AutoEncoder(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[8, 4, 4, 8], [4, 2, 2, 4], [2, 1, 1, 2]],
    "hidden_activation": ["relu", "sigmoid", "tanh"]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 83.6597   Inference Time: 28.2148   pAUC: 0.7329 ± 0.0007   Recall: 0.2646 ± 0.0099   TN: 465812.0   FP: 4680.0   FN: 72.0   TP: 27.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 79.8337   Inference Time: 28.5174   pAUC: 0.7322 ± 0.0010   Recall: 0.2667 ± 0.0049   TN: 465818.0   FP: 4674.0   FN: 72.0   TP: 27.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 78.0262   Inference Time: 25.9365   pAUC: 0.7330 ± 0.0005   Recall: 0.2687 ± 0.0049   TN: 465816.0   FP: 4676.0   FN: 72.0   TP: 27.0   Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Training 

# 2/6 - Features: session_duration, weekday

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 77.8120   Inference Time: 26.9456   pAUC: 0.6120 ± 0.0006   Recall: 0.1313 ± 0.0000   TN: 465800.0   FP: 4692.0   FN: 86.0   TP: 13.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 77.2154   Inference Time: 28.8283   pAUC: 0.6117 ± 0.0000   Recall: 0.1313 ± 0.0000   TN: 465800.0   FP: 4692.0   FN: 86.0   TP: 13.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 79.8061   Inference Time: 27.0901   pAUC: 0.6130 ± 0.0002   Recall: 0.1313 ± 0.0000   TN: 465797.0   FP: 4695.0   FN: 86.0   TP: 13.0   Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Training 

# 3/6 - Features: session_duration, hour, weekday

...

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 79.1327   Inference Time: 26.6911   pAUC: 0.6350 ± 0.0013   Recall: 0.0626 ± 0.0040   TN: 465787.0   FP: 4705.0   FN: 93.0   TP: 6.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 83.5392   Inference Time: 26.8909   pAUC: 0.6293 ± 0.0012   Recall: 0.0646 ± 0.0049   TN: 465796.0   FP: 4696.0   FN: 92.0   TP: 7.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 78.6108   Inference Time: 28.5132   pAUC: 0.6362 ± 0.0012   Recall: 0.0646 ± 0.0049   TN: 465792.0   FP: 4700.0   FN: 92.0   TP: 7.0   Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Training Tim

# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 80.1203   Inference Time: 26.4487   pAUC: 0.6170 ± 0.0013   Recall: 0.1313 ± 0.0000   TN: 465798.0   FP: 4694.0   FN: 86.0   TP: 13.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 83.6443   Inference Time: 29.1395   pAUC: 0.6175 ± 0.0015   Recall: 0.1313 ± 0.0000   TN: 465799.0   FP: 4693.0   FN: 86.0   TP: 13.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 82.0867   Inference Time: 28.5387   pAUC: 0.6162 ± 0.0005   Recall: 0.1313 ± 0.0000   TN: 465798.0   FP: 4694.0   FN: 86.0   TP: 13.0   Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Training 

# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 9 different hyperparameter combinations
 Training Time: 79.9268   Inference Time: 28.8382   pAUC: 0.6355 ± 0.0004   Recall: 0.0606 ± 0.0000   TN: 465795.0   FP: 4697.0   FN: 93.0   TP: 6.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 80.1600   Inference Time: 28.6435   pAUC: 0.6329 ± 0.0022   Recall: 0.0646 ± 0.0049   TN: 465788.0   FP: 4704.0   FN: 93.0   TP: 6.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 77.2241   Inference Time: 28.4431   pAUC: 0.6284 ± 0.0009   Recall: 0.0606 ± 0.0000   TN: 465792.0   FP: 4700.0   FN: 93.0   TP: 6.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
 Training Time: 63.5003   Inference Time: 25.8894   pAUC: 0.6362 ± 0.0006   Recall: 0.0606 ± 0.0000   TN: 465793.0   FP: 4699.0   FN: 93.0   TP: 6.0   Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 57.7694   Inference Time: 25.9969   pAUC: 0.6350 

# Training - Deep SVDD

In [ ]:
# Local configuration
classifier_name = "deep_svdd"

# Define the classifier that will be used for training
classifier = DeepSVDD(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[64, 32], [32, 16], [16, 8], [8, 4], [4, 2], [2, 1]],
    "hidden_activation": ["relu", "sigmoid", "tanh"],
    "use_ae": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: session_duration, hour

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 58.1550   Inference Time: 25.1021   pAUC: 0.7102 ± 0.0330   Recall: 0.2323 ± 0.0613   TN: 465814   FP: 4678    FN: 66    TP: 33    Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_563" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_563/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_563/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • y=a

# 2/6 - Features: session_duration, weekday

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 59.6773   Inference Time: 27.2280   pAUC: 0.6083 ± 0.0066   Recall: 0.1273 ± 0.0049   TN: 465801   FP: 4691    FN: 86    TP: 13    Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_1046" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_1046/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_1046/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • 

# 3/6 - Features: session_duration, hour, weekday

...

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 59.9379   Inference Time: 25.3450   pAUC: 0.6021 ± 0.0310   Recall: 0.0808 ± 0.0482   TN: 465803   FP: 4689    FN: 85    TP: 14    Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_2012" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_2012/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_2012/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • 

# 5/6 - Features: session_duration, weekday_sin, weekday_cos

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 60.6287   Inference Time: 27.5434   pAUC: 0.6134 ± 0.0103   Recall: 0.1333 ± 0.0040   TN: 465801   FP: 4691    FN: 85    TP: 14    Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_2495" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_2495/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_2495/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • 

# 6/6 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 63.6251   Inference Time: 26.2249   pAUC: 0.6046 ± 0.0224   Recall: 0.0909 ± 0.0482   TN: 465804   FP: 4688    FN: 84    TP: 15    Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_2978" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_2978/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_2978/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • 